## Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


## Loading Dataset

In [5]:
df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')

C:\Users\sathi\AppData\Local\Temp\ipykernel_5200\3370251438.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')


## Preprocessing Data

In [6]:
#Find all string values in sport
df['sport'] = df['sport'].astype(str)
#Replace '-' with 0
df['sport'] = df['sport'].replace('-', '80')
#COnvert hexadecimal values to decimal in sport
df['sport'] = df['sport'].apply(lambda x: int(x, 16))
#Convert sport to int
df['sport'] = df['sport'].astype(int)

len(df['sport'].unique())

43005

In [7]:
df['dsport'] = df['dsport'].astype(str)
df['dsport'] = df['dsport'].replace('-', '80')
df['dsport'] = df['dsport'].apply(lambda x: int(x, 16))
df['dsport'] = df['dsport'].astype(int)
len(df['dsport'].unique())


64380

In [8]:
dfAttack = df[df['Label']==1]
dfAttack.drop(['Label'], axis=1, inplace=True)
len(dfAttack)

C:\Users\sathi\AppData\Local\Temp\ipykernel_5200\4043762459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfAttack.drop(['Label'], axis=1, inplace=True)


321283

In [11]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,0,149.171.126.4,30306,114,5,0.119596,4550,68342,31,...,0,6,2,2,5,1,1,2,6,0
1,59.166.0.3,0,149.171.126.2,161545,114,5,0.650574,8928,320,31,...,0,3,5,2,4,1,1,4,6,0
2,59.166.0.8,0,149.171.126.9,20880,114,5,0.007980,2158,2464,31,...,0,3,5,1,1,1,1,3,6,0
3,149.171.126.18,0,175.45.176.3,83,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
4,149.171.126.18,0,175.45.176.3,83,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535847,59.166.0.5,0,149.171.126.7,0,114,5,0.087306,320,1828,31,...,0,1,2,3,3,1,1,3,6,0
2535848,59.166.0.7,0,149.171.126.4,0,114,2,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,6,0
2535849,59.166.0.3,0,149.171.126.9,0,114,2,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,6,0
2535850,59.166.0.9,0,149.171.126.0,0,114,2,2.200934,3498,166054,31,...,0,1,1,2,4,2,2,2,6,0


In [12]:
#Split train and test based on attack_cat
X = dfAttack.drop(['attack_cat'], axis=1)
y = dfAttack['attack_cat']
X = X.drop(['srcip','dstip'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Base Models

Logistic Regression

In [13]:
#Logistic regression to classify using all variables
from sklearn.metrics import roc_auc_score
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test,y_pred))
auc_scores = roc_auc_score(y_test, y_pred, multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
print("AUC Scores for each class:", auc_scores)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       558
           1       0.00      0.00      0.00       445
           2       0.00      0.00      0.00      3355
           3       0.47      0.54      0.51      8946
           4       0.66      0.04      0.07      4897
           5       0.79      0.99      0.88     42913
           7       0.00      0.00      0.00      2807
           8       0.00      0.00      0.00       298
           9       0.00      0.00      0.00        38

    accuracy                           0.74     64257
   macro avg       0.21      0.17      0.16     64257
weighted avg       0.64      0.74      0.66     64257



AxisError: axis 1 is out of bounds for array of dimension 1

In [16]:
#Count the number of each attack category in XTrain
y_train.value_counts()

5    172568
3     35579
4     19349
2     12998
7     11180
0      2119
1      1884
8      1213
9       136
Name: attack_cat, dtype: int64

In [17]:
y_test.value_counts()

5    42913
3     8946
4     4897
2     3355
7     2807
0      558
1      445
8      298
9       38
Name: attack_cat, dtype: int64

In [15]:
#Training score of Logistic Regression
print("Training score of Logistic Regression: ", lr.score(X_train, y_train))
#For each class
print("Accuracy of Logistic Regression: ", accuracy_score(y_test, y_pred))

Training score of Logistic Regression:  0.7402947561725273
Accuracy of Logistic Regression:  0.7371025724823754


Decision Tree

In [18]:
#Decision Tree and also visualise the tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))
print("Training score of Decision Tree: ", dt.score(X_train, y_train))
print("Accuracy of Decision Tree: ", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.61      0.18      0.28       558
           1       0.49      0.10      0.17       445
           2       0.33      0.36      0.34      3355
           3       0.64      0.76      0.69      8946
           4       0.95      0.87      0.91      4897
           5       1.00      0.99      0.99     42913
           7       0.89      0.77      0.83      2807
           8       0.85      0.83      0.84       298
           9       0.55      0.55      0.55        38

    accuracy                           0.89     64257
   macro avg       0.70      0.60      0.62     64257
weighted avg       0.90      0.89      0.89     64257

Training score of Decision Tree:  0.9304973037747154
Accuracy of Decision Tree:  0.8899730768632211


In [26]:
#Sort the Gini Index and find the most important and least important features
#With the feature names

importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 5 sbytes (0.622920)
2. feature 1 dsport (0.164337)
3. feature 11 service (0.037611)
4. feature 6 dbytes (0.021495)
5. feature 7 sttl (0.021233)
6. feature 21 dmeansz (0.017264)
7. feature 20 smeansz (0.016261)
8. feature 38 ct_srv_dst (0.012310)
9. feature 18 stcpb (0.011167)
10. feature 26 Stime (0.008336)
11. feature 9 sloss (0.005382)
12. feature 13 Dload (0.004789)
13. feature 37 ct_srv_src (0.004033)
14. feature 19 dtcpb (0.003968)
15. feature 31 synack (0.003931)
16. feature 32 ackdat (0.003838)
17. feature 12 Sload (0.003636)
18. feature 30 tcprtt (0.003404)
19. feature 4 dur (0.003202)
20. feature 25 Djit (0.003178)
21. feature 23 res_bdy_len (0.003127)
22. feature 28 Sintpkt (0.002834)
23. feature 24 Sjit (0.002834)
24. feature 27 Ltime (0.002691)
25. feature 29 Dintpkt (0.002579)
26. feature 10 dloss (0.002545)
27. feature 34 ct_flw_http_mthd (0.002181)
28. feature 40 ct_src_ ltm (0.001456)
29. feature 14 Spkts (0.001384)
30. feature 0 sport (0.001

## Dataset Sampling with Data Balancing

In [37]:
dfSample0 = df[df['Label'] == 0].sample(n=250000, random_state=42)
dfSample1 = df[df['Label'] == 1].sample(n=250000, random_state=42)

dfSample = pd.concat([dfSample0, dfSample1])
dfSample = dfSample.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset into features and labels
XSample = dfSample.drop('Label', axis=1)
ySample = dfSample['Label']

#Split the dataset into training and testing sets
XSample_train, XSample_test, ySample_train, ySample_test = train_test_split(XSample, ySample, test_size=0.2, random_state=42)

In [27]:
#Take equal number of samples for each label
dfDroppedforLR = df[df['Label'] == 0].sample(n=250000, random_state=40)
dfDroppedforLR = dfDroppedforLR.append(df[df['Label'] == 1].sample(n=250000, random_state=40))

#Drop srcip and dstip and use a new df
dfDroppedforLR = dfDroppedforLR.drop(['srcip', 'dstip','attack_cat','Stime','Ltime'], axis=1)

#Shuffle the dataset
dfDroppedforLR = dfDroppedforLR.sample(frac=1).reset_index(drop=True)

#Split the dataset into features and labels
X = dfDroppedforLR.iloc[:,0:42]
y = dfDroppedforLR['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\sathi\AppData\Local\Temp\ipykernel_5200\3429069213.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfDroppedforLR = dfDroppedforLR.append(df[df['Label'] == 1].sample(n=250000, random_state=40))


## Baseline Models with Sampled Dataset

Decision Tree

In [28]:
#Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))
print("Training score of Decision Tree: ", dt.score(X_train, y_train))
print("Accuracy of Decision Tree: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     49981
           1       0.99      0.99      0.99     50019

    accuracy                           0.99    100000
   macro avg       0.99      0.99      0.99    100000
weighted avg       0.99      0.99      0.99    100000

Training score of Decision Tree:  0.9998325
Accuracy of Decision Tree:  0.99047


In [29]:
#Sort the Gini Index and find the most important and least important features
#With the feature names

importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 31 ct_state_ttl (0.963163)
2. feature 1 dsport (0.005711)
3. feature 7 sttl (0.004860)
4. feature 20 smeansz (0.003025)
5. feature 5 sbytes (0.001611)
6. feature 19 dtcpb (0.001538)
7. feature 12 Sload (0.001507)
8. feature 35 ct_srv_src (0.001440)
9. feature 18 stcpb (0.001399)
10. feature 36 ct_srv_dst (0.001372)
11. feature 30 ackdat (0.001175)
12. feature 29 synack (0.001172)
13. feature 4 dur (0.001125)
14. feature 28 tcprtt (0.001073)
15. feature 25 Djit (0.001035)
16. feature 41 ct_dst_src_ltm (0.000907)
17. feature 26 Sintpkt (0.000867)
18. feature 24 Sjit (0.000838)
19. feature 27 Dintpkt (0.000773)
20. feature 38 ct_src_ ltm (0.000751)
21. feature 16 swin (0.000649)
22. feature 37 ct_dst_ltm (0.000606)
23. feature 13 Dload (0.000524)
24. feature 23 res_bdy_len (0.000513)
25. feature 39 ct_src_dport_ltm (0.000491)
26. feature 6 dbytes (0.000373)
27. feature 21 dmeansz (0.000368)
28. feature 0 sport (0.000333)
29. feature 40 ct_dst_sport_ltm (0.00023

PCA for sttl and ct_state_ttl with Logistic Regression

In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = X[['sttl', 'ct_state_ttl']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca_x_train = pca.fit_transform(X_train)
pca_x_test = pca.fit_transform(X_test)
classifier = LogisticRegression()
classifier.fit(pca_x_train, y_train)
y_pred = classifier.predict(pca_x_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.93      0.98      0.96     49981
           1       0.98      0.93      0.96     50019

    accuracy                           0.96    100000
   macro avg       0.96      0.96      0.96    100000
weighted avg       0.96      0.96      0.96    100000



In [33]:
#Linear equation formed by PCA
print("PCA Co efficients using Linear Equation of Logistic Regression: ", classifier.coef_)
print("PCA Co efficients for sttl and ct_state_ttl:")
print(pca.components_)

PCA Co efficients using Linear Equation of Logistic Regression:  [[-0.03420537]]
PCA Co efficients for sttl and ct_state_ttl:
[[-0.99996548 -0.00830924]]


PCA for sttl and ct_state_ttl with Decision Tree

In [44]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = X[['ct_state_ttl','sttl']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca_x_train = pca.fit_transform(X_train)
pca_x_test = pca.fit_transform(X_test)
classifier = DecisionTreeClassifier()
classifier.fit(pca_x_train, y_train)
y_pred = classifier.predict(pca_x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     49981
           1       0.98      1.00      0.99     50019

    accuracy                           0.99    100000
   macro avg       0.99      0.99      0.99    100000
weighted avg       0.99      0.99      0.99    100000



In [45]:
#Linear equation formed by PCA
print("PCA Co efficients for sttl and ct_state_ttl:")
print(pca.components_)

PCA Co efficients for sttl and ct_state_ttl:
[[-0.00830924 -0.99996548]]


## LSTM

# Proper class-wise dataset sampling and splitting

In [39]:
dfsample0 = dfSample0.sample(n=12500, random_state=42)
dfsample1 = dfSample1.sample(n=12500, random_state=42)

dfsample0train, dfsample0test = train_test_split(dfsample0, test_size=0.2, random_state=42)
dfsample1train, dfsample1test = train_test_split(dfsample1, test_size=0.2, random_state=42)

dfsampletrain = pd.concat([dfsample0train, dfsample1train])
dfsampletest = pd.concat([dfsample0test, dfsample1test])

XSample_train = dfsampletrain[['srcip','sport','dstip','dsport','sttl', 'ct_state_ttl']]
ySample_train = dfsampletrain['Label']
XSample_test = dfsampletest[['srcip','sport','dstip','dsport','sttl', 'ct_state_ttl']]
ySample_test = dfsampletest['Label']



dfsample = dfsampletrain 

# LSTM Dataset preprocessing

In [41]:
nodes = set()
edges = set()
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)
    edges.add((src,dst))
len(nodes)

2995

In [42]:
len(edges)

3988

In [43]:
2995*2995

8970025

In [46]:
import numpy as np
# Prepare an adjacency matrix of the nodes
adjacency_matrix = np.zeros((len(nodes), len(nodes)), dtype=float)
label_matrix = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    adjacency_matrix[src_index, dst_index] = 0.99996548*dfsample['sttl'].iloc[i] + 0.00830924*dfsample['ct_state_ttl'].iloc[i]
    label_matrix[src_index, dst_index] = dfsample['Label'].iloc[i]
# Flatten tuple values in the adjacency matrix
adjacency_matrix_flat = adjacency_matrix.reshape(-1, 1)
label_flat = label_matrix.reshape(-1, 1) 


# LSTM MODEL TRAINING FOR edge_weight = 0.99996548 * sttl + 0.00830924 * ct_state_ttl (DONOT RUN)

In [49]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Modify the call method of the LSTMClassifier
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(LSTMClassifier, self).__init__()
        self.lstm = LSTM(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.lstm(x)
        output = self.dense(x)
        return output



# Hyperparameters
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification

# Convert adjacency_matrix_flat to a TensorFlow tensor
adjacency_tensor = tf.convert_to_tensor(adjacency_matrix_flat, dtype=tf.float32)

# Initialize model, optimizer, and loss function
model = LSTMClassifier(input_shape, hidden_units, output_units)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Training
history = model.fit(adjacency_tensor, label_flat, epochs=2, batch_size=32, validation_split=0.2)

# Evaluation
#loss, accuracy = model.evaluate(adjacency_tensor, label_flat) 
#print(f'Loss: {loss}, Accuracy: {accuracy}')





Epoch 1/2


224251/224251 [==============================] - 2646s 12ms/step - loss: 8.5013e-04 - accuracy: 1.0000 - val_loss: 1.1908e-05 - val_accuracy: 1.0000
Epoch 2/2
224251/224251 [==============================] - 2072s 9ms/step - loss: 5.4401e-05 - accuracy: 1.0000 - val_loss: 1.0904e-05 - val_accuracy: 1.0000


Saving the LSTM Model weights

In [51]:
# Save the model to HDF5 file
model.save_weights("lstm_weights_sttl_ct_state_ttl_pca.h5")

# Load the weights to create the LSTM for edge_weight = 0.99996548 * sttl + 0.00830924 * ct_state_ttlfrom stored data

LSTM needs some dummy data to get initialised

In [61]:
# Create an instance of the model
# Hyperparameters
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Modify the call method of the LSTMClassifier
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(LSTMClassifier, self).__init__()
        self.lstm = LSTM(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.lstm(x)
        output = self.dense(x)
        return output
    
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification
# Initialize model, optimizer, and loss function
model_test_load = LSTMClassifier(input_shape, hidden_units, output_units)

dummy_input = np.zeros((4, 4), dtype=float)
dummy_label = np.zeros((4, 4))
#Use dummy data to initialize the model
dummy_flat = dummy_input.reshape(-1, 1)
dummy_label_flat = dummy_label.reshape(-1, 1) 

model_test_load.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_test_load.fit(dummy_flat, dummy_label_flat, epochs=2, batch_size=32, validation_split=0.2)


Epoch 1/2
1/1 [==============================] - 31s 31s/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6921 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 269ms/step - loss: 0.6921 - accuracy: 1.0000 - val_loss: 0.6910 - val_accuracy: 1.0000


In [63]:
# Load the saved weights
model_test_load.load_weights("lstm_weights_sttl_ct_state_ttl_pca.h5")

Preparing Testing dataset

In [64]:

nodes = set()
edges = set()
for i in range(len(dfsampletest)):
    src = str(dfsampletest['srcip'].iloc[i])+':'+str(dfsampletest['sport'].iloc[i])
    dst = str(dfsampletest['dstip'].iloc[i])+':'+str(dfsampletest['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)
    edges.add((src,dst))


In [65]:
len(nodes)

915

In [66]:
len(edges)

1422

In [67]:
import numpy as np
# Prepare an adjacency matrix of the nodes
adjacency_matrix_test = np.zeros((len(nodes), len(nodes)), dtype=float)
label_matrix_test = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsampletest)):
    src = str(dfsampletest['srcip'].iloc[i])+':'+str(dfsampletest['sport'].iloc[i])
    dst = str(dfsampletest['dstip'].iloc[i])+':'+str(dfsampletest['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    adjacency_matrix_test[src_index, dst_index] = 0.99996548*dfsampletest['sttl'].iloc[i] + 0.00830924*dfsampletest['ct_state_ttl'].iloc[i]
    label_matrix_test[src_index, dst_index] = dfsampletest['Label'].iloc[i]
# Flatten tuple values in the adjacency matrix
adjacency_matrix_test_flat = adjacency_matrix_test.reshape(-1, 1)
label_test_flat = label_matrix_test.reshape(-1, 1)

In [68]:
predictions = model.predict(adjacency_matrix_test_flat)
predictionsclass = (predictions > 0.5).astype(int)

26164/26164 [==============================] - 146s 5ms/step


In [75]:
predictions_test = model_test_load.predict(adjacency_matrix_test_flat)
predictionsclass_test = (predictions_test > 0.5).astype(int)

25116/26164 [===========================>..] - ETA: 6s

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictionsclass_test)

print("Classification Report:")
print(report)

In [69]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictionsclass)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    836839
         1.0       0.92      1.00      0.96       386

    accuracy                           1.00    837225
   macro avg       0.96      1.00      0.98    837225
weighted avg       1.00      1.00      1.00    837225



In [70]:
#AuC for each class
from sklearn.metrics import roc_auc_score
roc_auc_score(label_test_flat, predictions_test, multi_class='ovr')


0.999985708308815

In [73]:
auc_score = roc_auc_score(label_test_flat, predictions_test)

print("AUC Score for Positive Class (Label 1):", auc_score)

AUC Score for Positive Class (Label 1): 0.999985708308815


In [74]:
predictions_label_0 = 1 - predictions_test
labels_label_0 = 1 - label_test_flat

# Calculate AUC score for label = 0
auc_score_label_0 = roc_auc_score(labels_label_0, predictions_label_0)

print("AUC Score for Label 0:", auc_score_label_0)

AUC Score for Label 0: 0.999985708308815
